Import all necessary modules

In [1]:
import math
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import csv

Function Dictionary

In [2]:
# Calculate E(z), which in this situation depends on w
def E(z_i, w, omega_m, omega_l):
    omega_k = 1.0 - omega_m - omega_l
    E = np.sqrt(omega_m * (1.0 + z_i)**3 + omega_k * (1.0 + z_i)**2 + omega_l * (1.0+z_i)**(3.0*(1.0+w)))
    return E;

# Calculate the integral D_C/D_H, store it in an array
def DCDH_int(z_limit, w, omega_m, omega_l):
    f = lambda z_i: 1.0 / E(z_i, w, omega_m, omega_l)
    i = integrate.quad(f, 0, z_limit)
    return i[0]

In [3]:
# Proper Motion distance integrals

# proper motion distance for omega_l == 0
def prop_motion_0(z, w, DMDH, omega_m, omega_l):
    # create the output array DMDH
    DMDH = np.copy(z);

    # find omega_k = 1 - omega_m - omega_l
    omega_k = 1.0 - omega_m - omega_l
    print(r"$\Omega_k =$ " + str(omega_k))
    print(r"$\Omega_\Mu =$ " + str(omega_m))
    print(r"$\Omega_\Lambda =$ " + str(omega_l))

    # Calculate D_M/D_H for different universe geometries
    # open universe (omega_k > 0)
    if (omega_k > 1.0e-6):
        for i in range(len(z)):
            DMDH[i] = 1/np.sqrt(omega_k) * np.sinh( np.sqrt(omega_k) * DCDH_int(z[i],w,omega_m,omega_l) )
    # closed universe (omega_k < 0)
    elif (omega_k < -1.0e-6):
        for i in range(len(z)):
            DMDH[i] = 1/np.sqrt(abs(omega_k)) * np.sin( np.sqrt(abs(omega_k)) * DCDH_int(z[i],w,omega_m,omega_l) )
    # flat universe (omega_k = 0)
    else:
        DMDH = 2.0*( 2.0 - omega_m*(1.0-z) - (2-omega_m)*np.sqrt(1.0+omega_m*z) ) / ( omega_m*omega_m*(1.0+z) );

    return DMDH

# proper motion distance for omega_l != 0
def prop_motion(z, w, DMDH, omega_m, omega_l):
    DMDH = np.copy(z);
    
    # find omega_k = 1 - omega_m - omega_l
    omega_k = 1.0 - omega_m - omega_l
    print(omega_k)
    
    # Calculate D_M/D_H for different universe geometries
    if (omega_k > 1.0e-6):
        for i in range(len(z)):
            DMDH[i] = 1/np.sqrt(omega_k) * np.sinh( np.sqrt(omega_k) * DCDH_int(z[i],w,omega_m,omega_l) )
    # closed universe
    elif (omega_k < -1.0e-6):
        for i in range(len(z)):
            DMDH[i] = 1/np.sqrt(abs(omega_k)) * np.sin( np.sqrt(abs(omega_k)) * DCDH_int(z[i],w, omega_m,omega_l) )
    # flat universe
    else:
        for i in range(len(z)):
            DMDH[i] = DCDH_int(z[i], w, omega_m, omega_l)
    return DMDH

In [4]:
# angular diameter distance
def dist_angdiam(z, DMDH, DADH, omega_m, omega_l):
    DADH = np.copy(z);
    DADH = DMDH / (1.0+z)
    return DADH;

# luminosity distance
def dist_lum(z, DMDH, DLDH, omega_m, omega_l):
    DLDH = np.copy(z)
    DLDH = DMDH * (1.0+z)
    return DLDH;

# distance modulus
def dist_mod(z, D_H, omega_m, omega_l, DMDH):
    # initialize the distance modulus array
    mu = np.copy(z)

    # convert D_H from Mpc to pc
    D_H = D_H * 1.0e6

    # calculate each value of mu array
    #for i in range(len(z)):
        #mu[i] = 5.0 * ( np.log10(1.0+z[i]) + np.log10(DCDH_int(z[i], omega_m, omega_l)) + np.log10(D_H/10) )
    mu = 5.0 * ( np.log10(1.0+z) + np.log10(DMDH) + np.log10(D_H/10) )
    return mu;

# comoving volume element
def com_vol(z, DMDH, DVC, omega_m, omega_l, D_H):
    DVC = np.copy(z)
    for i in range(len(z)):
        DVC[i] = DMDH[i]**2 * D_H**3 / ( E(z[i], omega_m, omega_l) )
    return DVC

Variable Dictionary

In [5]:
# Constants
omega_k = 0                     # critical density
c = 2.9979e5                    # speed of light (km/s)
omega_m = None                  # mass density parameter
w = None                        # equation of state
omega_l = None                  # energy density
pressure = 0.0                  # pressure of the universe

# Variable dictionary
H = None                        # Hubble constant (km/s/Mpc)
z = None                        # redshift
D_C = None                      # comoving distance, line-of-sight (m)
D_M = None                      # comoving distance, transverse (m)
D_A = None                      # angular diameter distance (m)
D_L = None                      # luminosity distance (m)
V_C = None                      # comoving volume (m^3)
D_H = None                      # Hubble Distance (Mpc)
array_size = None               # size of each array, which is the number of z values
param_text = ""                 # label for the plots
npts = None                     # Number of points (omega_m, w) we use

# Output arrays
z = None;                       # Array of input values of z
DMDH = np.array([])             # Array of output values of D_M/D_H
DADH = np.array([])             # Array of output values of D_A/D_H
DLDH = np.array([])             # Array of output values of D_L/D_H
dmod = np.array([])             # distance modulus array
DVC = np.array([])              # comoving volume array
tLtH = np.array([])             # Array of lookback times
age = np.array([])              # Array of ages
dPdz = np.array([])             # dimensionless intersection probability

We draw arrays of corresponding density parameters from Figure 20.

We note that equation of state $w$ and energy density $\Omega_\Lambda$ are related as follows:
$$w = \frac{pressure}{\Omega_\Lambda}$$

Take in values. Test each value of omega_m and w individually.

In [6]:
# arrays of density parameters

# Take in densities
print("\u03A9_k = 0.0")     # omega_k = 0.0

# Take in Hubble constant
H = 70
D_H = c / H                 # Hubble distance

# Take in w-values and omega_m values
omega_m = float(input("Value of omega_m: "))
w = float(input("Value of w: "))

# Parameters for plot, including the range of redshift values and y-axis
zmin = 0.001
zmax = 2.5
ymin = 0
ymax = 50

# Define the array of z-values (redshift)
z = np.linspace(zmin, zmax, 100)

Ω_k = 0.0
Value of omega_m: 0.5
Value of w: -1.0


Plot the Distance Modulus. Assume a flat universe, i.e. $\Omega_\Lambda = 1 - \Omega_M$.

In [14]:
# Begin by collecting csv data
sne_x = []
sne_y = []
err_sne_y = []

with open('ps1_data.txt', newline='') as csvfile:
    ps1_data = csv.reader(csvfile, delimiter=' ')
        
    i = 0
    for row in ps1_data:
        if (i > 0):
            sne_x.append(float(row[1]))
            sne_y.append(float(row[4]))
            err_sne_y.append(float(row[5]))
        i+= 1
csvfile.close()

# convert all lists to arrays
sne_x = np.array(sne_x)
sne_y = np.array(sne_y)
err_sne_y = np.array(err_sne_y)

# Calculate the distance moduli and plot them
omega_l = 1.0 - omega_m
if (omega_l == 0):
    DMDH = prop_motion_0(z, w, DMDH, omega_m, omega_l)
else:
    DMDH = prop_motion(z, w, DMDH, omega_m, omega_l)

mu = dist_mod(z, D_H, omega_m, omega_l, DMDH)


0.0


Generate a residual plot by feeding values from sne_x into the prop_motion() function.

In [8]:
# test values of omega_m and omega_l

# Feed the sne_x array into the functions
if (omega_l == 0):
    DMDH = prop_motion_0(sne_x, w, DMDH, omega_m, omega_l)
else:
    DMDH = prop_motion(sne_x, w, DMDH, omega_m, omega_l)

# corresponding predicted distance modulus
dm_predicted = dist_mod(sne_x, D_H, omega_m, omega_l, DMDH)

# actual distance modulus
dm_actual = np.copy(sne_y)

# calculate residuals and convert to array
res = dm_actual - dm_predicted

# Find the average residual value
avg_res = np.mean(res)
# Plot a horizontal line corresponding to y = avg_res
avg_y = np.copy(z)    # Create a copy of the z-array into the avg_y array
for i in range(len(avg_y)):    # Assign each slot of avg_y the value avg_res
    avg_y[i] = avg_res

# re-calculate the error bars for residuals if needed
# sigma_res = sqrt(sigma_dm_actual^2 + sigma_dm_predicted^2)
err_predicted = 0.0    # assume zero uncertainty in the prediction function
sigma_res = np.array([])
sigma_res = err_sne_y**2 + err_predicted**2
sigma_res = np.sqrt(sigma_res)

0.0


In [12]:
# Calculate chi-square
adj_res = res-avg_res    # adjusted residual, to be centered on 0
chi2 = 0.0               # chi-square value

for j in range(len(res)):
    chi2 += (adj_res[j]*adj_res[j]) / (sigma_res[j]*sigma_res[j])

# Print the chi-square value    
print("chi-square: " + str(chi2))

# Print the parameters used to make chi-square
print("Omega_m=" + str(omega_m))
print("w=" + str(w))
print("Offset=" + str(avg_res))

chi-square: 1264.8879581570397
Omega_m=0.5
w=-1.0
Offset=-19.257025509750704


Examples of good data points $(\Omega_M, w)$:

(0.2,-0.8)

(0.3,-1.0)

(0.4,-1.3)

Examples of bad data points $(\Omega_M, w)$:

(0.5, -1.0)

(0.25, -0.2)

(0.3, -2.0)

(0.1, -1.5)